In [25]:
import numpy as np 
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
import cvxopt
import cvxopt.solvers

In [26]:
"""
this is the code you need to run to import the data. 
You just have to change line 40 putting the correct path.
"""

train = pd.read_csv('fashion-mnist_train.csv')
test = pd.read_csv('fashion-mnist_test.csv')

indexes = [2,3]

train_set = train[train['label'].isin(indexes)]
test_set = test[test['label'].isin(indexes)]

X_Train = train_set.values[:,1:]
Y_Train = train_set.values[:,0]

X_Test = test_set.values[:,1:]
Y_Test = test_set.values[:,0]

ind_2 = np.where(Y_Train==2)
ind_2_Test = np.where(Y_Test==2)

ind_3 = np.where(Y_Train==3)
ind_3_Test = np.where(Y_Test==3)

X2 = X_Train[ind_2[0][:1000]]
Y2 = np.ones(1000)

X3 = X_Train[ind_3[0][:1000]]
Y3 = -np.ones(1000)

X2test = X_Test[ind_2_Test[0][:200]]
Y2test = np.ones(X2test.shape[0])

X3test = X_Test[ind_3_Test[0][:200]]
Y3test = -np.ones(X3test.shape[0])


X_Train = np.concatenate((X2,X3))
Y_Train = np.concatenate((Y2,Y3))


X_Test = np.concatenate((X2test,X3test))
Y_Test = np.concatenate((Y2test,Y3test))

In [27]:
print(X_Train.shape)
print(X_Test.shape)
print(Y_Train.shape)
print(Y_Test.shape)

(2000, 784)
(400, 784)
(2000,)
(400,)


In [28]:
Y_Train = Y_Train.astype(int)

In [29]:
# Check oversampling
k = 0
j = 0
for i in Y_Train:
    if i == 1:
        k += 1
    else:
        j += 1
print(k)
print(j)

1000
1000


In [30]:
def scale_dataset(X, y, oversample=False):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
     
    if oversample:
        ros = RandomOverSampler()
        X, y = ros.fit_resample(X, y)

    data = np.hstack((X, np.reshape(y, (len(y), 1))))
    
    return data, X, y

In [31]:
train, x_train, y_train = scale_dataset(X_Train, Y_Train)
test, x_test, y_test = scale_dataset(X_Test, Y_Test)

In [32]:
y_train = y_train.astype(float)
y_test = y_test.astype(float)

In [33]:
# K_fold cross validation
def k_fold_cross_validation(data, num_folds):
    fold_size = len(data) // num_folds
    fold_indices = np.arange(len(data))
    np.random.seed(2108602)
    np.random.shuffle(fold_indices)

    folds_data = {}

    for i in range(num_folds):
        validation_indices = fold_indices[i * fold_size : (i + 1) * fold_size]
        train_indices = np.concatenate([fold_indices[:i * fold_size], fold_indices[(i + 1) * fold_size:]])

        folds_data[i] = {
            'train': data[train_indices],
            'validation': data[validation_indices]
        }

    return folds_data

In [34]:
# Kernel Functions
def polynomial_kernel(x, y, p = 3):
    return (1 + np.dot(x, y)) ** p

 
def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2))) 

In [35]:
def fit(x, y, C, sigma, p=0, gass = True , itr=1000):  
    n_samples, n_features = x.shape
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            if gass == True:
                K[i,j] = gaussian_kernel(x[i], x[j], sigma)
            else:
                K[i,j] = polynomial_kernel(x[i], x[j], p)


    A = cvxopt.matrix(y_train, (1, n_samples))
    P = cvxopt.matrix(np.outer(y,y) * K)
    q = cvxopt.matrix(np.ones(n_samples) * -1)
    b = cvxopt.matrix(0.0)
    G = cvxopt.matrix(np.vstack((np.eye(n_samples) * -1, np.eye(n_samples))))
    h = cvxopt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C))) 

    # solve QP problem
    cvxopt.solvers.options['maxiters'] = itr
    cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)

    # Lagrange multipliers
    a = np.ravel(solution['x'])

    return solution, a, K

In [36]:
# Support vectors have non zero lagrange multipliers
def SV(a, x, y, K):
    sv = a > 1e-05
    ind = np.arange(len(a))[sv]
    a = a[sv]
    sv_x = x[sv]
    sv_y = y[sv]

    # Intercept calculation
    """b = np.sum(sv_y)
    b -= np.sum(a * sv_y * K[sv][:, sv])
    b /= len(a)"""
    b_sum = 0
    for n in range(len(a)):
        b_sum += sv_y[n]
        b_sum -= np.sum(a * sv_y * K[ind[n], sv])
    b = b_sum / len(a)

    # Weight calculation
    w = np.dot(sv_x.T, a * sv_y)

    return a, b, w

In [37]:
def predict(X, w, b):
    return np.sign(np.dot(X, w) + b) 

In [38]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [39]:
'''sigma = [5, 7, 9, 12, 15] # 1e-3 : 1
C = [0.1, 1, 5, 7, 9] 
n_folds = 5
folds = k_fold_cross_validation(train, n_folds)

for i in np.arange(len(folds)):
    print("Fold number: ", i)
    k_t = folds[i]['train']
    k_v = folds[i]['validation']
    x_train = k_t[:, :-1]
    y_train = k_t[:, -1]
    x_valid = k_v[:, :-1]
    y_valid = k_v[:, -1]
    
    for sig in sigma:
        for c in C:
            solution, a, K = fit(x_train, y_train, c, sig) 
            a, b, w = SV(a, x_train, y_train, K)  
            y_pred = predict(x_valid, w, b)
            print(f"Sigma: {sig}, C: {c}, # SV: {len(a)}, Accuracy: {accuracy(y_valid, y_pred)}")
'''

'sigma = [5, 7, 9, 12, 15] # 1e-3 : 1\nC = [0.1, 1, 5, 7, 9] \nn_folds = 5\nfolds = k_fold_cross_validation(train, n_folds)\n\nfor i in np.arange(len(folds)):\n    print("Fold number: ", i)\n    k_t = folds[i][\'train\']\n    k_v = folds[i][\'validation\']\n    x_train = k_t[:, :-1]\n    y_train = k_t[:, -1]\n    x_valid = k_v[:, :-1]\n    y_valid = k_v[:, -1]\n    \n    for sig in sigma:\n        for c in C:\n            solution, a, K = fit(x_train, y_train, c, sig) \n            a, b, w = SV(a, x_train, y_train, K)  \n            y_pred = predict(x_valid, w, b)\n            print(f"Sigma: {sig}, C: {c}, # SV: {len(a)}, Accuracy: {accuracy(y_valid, y_pred)}")\n'

In [40]:
n_folds = 5
folds = k_fold_cross_validation(train, n_folds)
k_t = folds[4]['train']
k_v = folds[4]['validation']
x_train = k_t[:, :-1]
y_train = k_t[:, -1]

In [41]:
sigma = 30  # 15
C = 1     # 0.1
solution, a, K = fit(x_train, y_train, C, sigma)
alpha = a 
a, b, w = SV(a, x_train, y_train, K)  
y_pred = predict(x_test, w, b)
print(f"Accuracy: {accuracy(y_test, y_pred)}") 

Accuracy: 0.975


In [42]:
print(max(alpha))
print(min(alpha))
print(np.average(alpha))

0.9999999998086938
2.762834761041612e-10
0.11712044087830105


In [43]:
u = alpha >= 1e-04
l = alpha < 1e-04
U = alpha[u]
L = alpha[l] 

In [44]:
print(L.shape)
print(U.shape)

(1361,)
(239,)


In [45]:
def Kernel(x, y, sigma):
    n_samples, n_features = x.shape
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K[i,j] = gaussian_kernel(x[i], x[j], sigma)
    return K
def kkt_check(alpha, x_train, y_train): 
    u = alpha >= 0.000999
    l = alpha < 1e-04
    U = alpha[u]
    L = alpha[l]
    U_y = y_train[u]
    U_x = x_train[u]
    L_y = y_train[l]
    L_x = x_train[l]
    U_ps = U[np.where(U_y > 0)]
    U_ng = U[np.where(U_y < 0)]
    L_ps = L[np.where(L_y > 0)]
    L_ng = L[np.where(L_y < 0)]


    R = np.hstack((L_ps, U_ng))
    y_R = np.hstack(( U_y[np.where(U_y<0)], L_y[np.where(L_y>0)]))
    x_R = np.vstack((U_x[np.where(U_y<0)], L_x[np.where(L_y>0)]))

    S = np.hstack((L_ng, U_ps))
    y_S = np.hstack((U_y[np.where(U_y>0)], L_y[np.where(L_y<0)]))
    x_S = np.vstack((U_x[np.where(U_y>0)], L_x[np.where(L_y<0)]))

        
    K_R = Kernel(x_R, x_R, sigma)
    K_S = Kernel(x_S, x_S, sigma)

    P_R = np.outer(y_R, y_R) * K_R
    P_S= np.outer(y_S, y_S) * K_S

    df_R = np.dot(P_R, R.T) - np.ones(len(R))
    df_S = np.dot(P_S, S.T) - np.ones(len(S))

    m = -y_R*df_R
    M = -y_S*df_S

    if max(m) <= min(M):
        print('Optimal')
    else:
        print('Not_Optimal')

In [46]:
kkt_check(alpha, x_train, y_train)

Optimal
